# Import des outils / jeu de données

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
df_transforme = pd.read_csv(
    "data/data-transformed.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "Complain",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

# Régressions

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)

In [ ]:
X = pd.get_dummies(df_transforme.drop(columns=["Response"]))

In [ ]:
y = df[["Response"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.head()

In [ ]:
log_reg = LogisticRegression(random_state=0)

In [ ]:
log_reg.fit(X_train, y_train)

In [ ]:
y_pred = log_reg.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, labels=log_reg.classes_))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=log_reg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=log_reg.classes_)
disp.plot()

In [ ]:
# todo: commenter

## Régression PLS

In [ ]:
from sklearn.cross_decomposition import PLSRegression

### Modèle d'ajustement

In [ ]:
# Initialisation de l'objet PLSRegression avec 2 composantes PLS
pls = PLSRegression(n_components=2)

In [ ]:
# Apprentissage du modèle sur les données
pls.fit(df_transforme[var_numeriques], df_transforme["NumStorePurchases"])

In [ ]:
# Prédiction de la variable cible sur de nouvelles données
y_pred = pls.predict(df_transforme[var_numeriques])

In [ ]:
# Evaluation de la performance du modèle
r2 = pls.score(
    df_transforme[var_numeriques], df_transforme["NumStorePurchases"].astype(int)
)

In [ ]:
r2

### Modèle de prédiction

In [ ]:
X = df_transforme[var_numeriques].drop(columns=["NumStorePurchases"])

In [ ]:
y = df[["NumStorePurchases"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Initialisation de l'objet PLSRegression avec 2 composantes PLS
pls = PLSRegression(n_components=3)

In [ ]:
# Apprentissage du modèle sur les données
pls.fit(X_train, y_train)

In [ ]:
# Prédiction de la variable cible sur de nouvelles données
y_pred = pls.predict(X_test)

In [ ]:
# Evaluation de la performance du modèle
pls.score(X_test, y_test)

In [ ]:
print(f"MSE = {mean_squared_error(y_test, y_pred)}")
print(f"RMSE = {mean_squared_error(y_test, y_pred, squared=False)}")
print(f"MAE = {mean_absolute_error(y_test, y_pred)}")

## Régression linéaire simple

Modèle simple : une variable à expliquer $Y$ et une seule variable explicative $X$.

$$y_i = \beta_0 + \beta_1 X_i + \epsilon_i$$

Hypothèses à vérifier pour la régression linéaire simple :

1) il existe une corrélation linéaire entre $X$ et $Y$

1) la distribution de l’erreur $\epsilon$ est indépendante de la variable X (exogénéité)

2) l'erreur suit une loi normale centrée i.e. $E(ε_i) = 0$

3) l’erreur est de variance constante (homoscédasticité)
i.e $Var(ε_i) = \sigma^2$, une constante

4) les erreurs sont indépendantes (absence d'autocorrélation)
i.e. $Cov(εi, εj) = 0$, pour tout $i, j \in N \times N, i \neq j$

In [ ]:
X = np.array(df_transforme["Income"])
Y = np.array(df_transforme["NumStorePurchases"])

### Hypothèse 1 : corrélation linéaire

In [ ]:
from scipy.stats import pearsonr

In [ ]:
print(
    "Le coefficient de corrélation linéaire entre X et Y vaut",
    pearsonr(X, Y)[0],
    "la pvalue associée vaut",
    pearsonr(X, Y)[1],
    "il existe donc bien une relation linéaire entre X et Y.",
)

Le reste des hypothèses à tester requiert d'effectuer la régression linéaire :

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
model = LinearRegression().fit(X_train.reshape(-1, 1), Y_train)
Y_train_hat = model.predict(X_train.reshape(-1, 1))

In [ ]:
plt.scatter(X_train, Y_train, color="black")
plt.plot(X_train, Y_train_hat, color="red")
plt.title("Régression linéaire du nombre d'achats en magasin en fonction du revenu")

In [ ]:
print(model.intercept_, model.coef_, model.score(X_train.reshape(-1, 1), Y_train))

Equation de régression :

$$y_i = 0.46 + 2.21 * x_i$$

### Hypothèse 2 : exogénéité

In [ ]:
residuals = Y_train - Y_train_hat

plt.scatter(X_train, residuals)

In [ ]:
print(
    "Le coefficient de corrélation entre X et les residus vaut",
    pearsonr(X_train, residuals)[0],
    ". On a bien indépendance et donc exogénéité.",
)

### Hypothèse 3 : l'erreur suit une loi normale centrée i.e. E(ε_i) = 0

In [ ]:
import statistics
from scipy import stats

In [ ]:
plt.hist(residuals, density=True)

x = np.linspace(-7, 7, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))
plt.title("Histogramme des résidus en superposition avec la densité de la loi normale")
plt.show()

In [ ]:
print("La moyenne des résidus vaut", statistics.mean(residuals))
print("Mais les résidus ne suivent pas une loi normale")

In [ ]:
sm.qqplot(residuals, line="45")
print("On constate sur le qqplot que les points ne suivent pas la droite x = y")

In [ ]:
print(
    "Un test de shapiro, pour tester l'hypothèse de normalité, nous donne une pvalue de",
    stats.shapiro(residuals)[1],
    ". On rejette l'hypothèse nulle et on conclut que les résidus ne suivent pas une loi normale.",
)

### Hypothèse 4 : homoscédasticité

In [ ]:
def abline(slope, intercept):
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, "-", color="red")

In [ ]:
plt.plot(residuals, "bo")
plt.title("Nuage de points des résidus")
abline(0, 7)
abline(0, -7)

In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

In [ ]:
fit = sm.OLS(Y_train, sm.add_constant(X_train)).fit()

In [ ]:
fit.summary()

In [ ]:
res_names = ["F statistic", "p-value"]
gq_test = sms.het_goldfeldquandt(fit.resid, fit.model.exog)
lzip(res_names, gq_test)

In [ ]:
bp_test = sms.het_breuschpagan(fit.resid, fit.model.exog)
lzip(res_names, bp_test)

Le test de Goldfeld-Quandt ne nous donne pas d'hétéroscédasticité, contraiement au test de Breusch-Pagan.

### Hypothèse 5 : absence d'autocorrélation

In [ ]:
from statsmodels.graphics.regressionplots import *

In [ ]:
sm.graphics.tsa.plot_acf(residuals)
print("On remarque une absence d'autocorrélation.")

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X_train, cooks[0])
plt.xlabel("Revenus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.005) if x]
print(cooks_indexes)
# affiche les indexes des individus dont les distances de cook dépassent une certaine valeur

In [ ]:
influence_plot(fit)
print("")

### Score du modèle

#### Qualité d'ajustement

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
print(f"Le R² du modèle vaut {model.score(X_train.reshape(-1, 1), Y_train)}")

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model.score(X_train.reshape(-1, 1), Y_train)}"
)
print(
    f"R² du modèle sur les données de test = {model.score(X_test.reshape(-1, 1), Y_test)}"
)

In [ ]:
Y_test_hat = model.predict(X_test.reshape(-1, 1))

In [ ]:
plt.scatter(X_test, Y_test, color="black")
plt.plot(X_test, Y_test_hat, color="red")
plt.title("Nombre d'achats en magasin en fonction du revenu : données test")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
# todo: cross-validation

## Régression linéaire multiple

Modèle multiple : une variable à expliquer $Y$ et $P$ variables explicatives $X_i$

$$y_i = \beta_0 + \beta_1 X_i(1) + \beta_2 X_i(2) + \beta_3 X_i(3) + ... + \beta_P X_i(P) + \epsilon_i$$

Hypothèses à vérifier pour la régression linéaire multiple :

1) il existe une corrélation linéaire entre $X_i$ et $Y$

2) $Cov(X_i, ε_j) = 0$, pour tout $i, j \in N \times N, i \neq j$ (exogénéité) et pour chaque variable explicative $X_p$

3) l'erreur suit une loi normale centrée i.e. $E(ε_i) = 0$

4) l’erreur est de variance constante (homoscédasticité)
i.e $Var(ε_i) = \sigma^2$, une constante

5) les erreurs sont indépendantes (absence d'autocorrélation)
i.e. $Cov(εi, εj) = 0$, pour tout $i, j \in N \times N, i \neq j$

6) absence de colinéarité entre les variables explicatives,
i.e. $X^t * X$ est régulière, $det(X^t * X) \neq 0$

Pour les besoins de la régression linéaire, nous créons des variables muettes (dummy variables) pour inclure les variables catégoriques `Education` et `Marital_status` à la régression.

In [ ]:
df_reg = df_transforme.copy()

In [ ]:
df_reg = pd.get_dummies(df_reg, columns=["Education", "Marital_Status"])

In [ ]:
df_reg.columns

Pour cette régression, on suppose que l'entreprise veut profiler au mieux les clients qui achètent dans le magasin. C'est pourquoi la variable d'intérêt pour la régression sera le `nombre d'achats en magasin`.

Nos variables explicatives sont des variables numériques et des variables catégoriques transformées en variables muettes.
Nous commençons par un grand nombre de variables explicatives puis nous en éliminerons progressivement pendant la vérifications des hypothèses.

In [ ]:
var_numeriques_reg = [
    "Income",
    "Year_Birth",
    "Recency",
    "NbAcceptedCampaigns",
    "NumWebPurchases",
    "NumDealsPurchases",
    "NumWebVisitsMonth",
    "NumCatalogPurchases",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
]

var_categoriques_reg = [
    "NbChildren",
    "Education_2n Cycle",
    "Education_Basic",
    "Education_Graduation",
    "Education_Master",
    "Education_PhD",
    "Marital_Status_Divorced",
    "Marital_Status_Married",
    "Marital_Status_Single",
    "Marital_Status_Together",
    "Marital_Status_Widow",
]

In [ ]:
X = df_reg[var_categoriques_reg + var_numeriques_reg]
Y = df_reg["NumStorePurchases"]

In [ ]:
print(
    "Nous avons",
    X.shape[1],
    "variables explicatives au début de la régression dont",
    X[var_numeriques_reg].shape[1],
    "variables numériques.",
)

### Hypothèse 1 : lien linéaire entre Y et les variables explicatives numériques

In [ ]:
sns.pairplot(df_transforme, x_vars=var_numeriques_reg[0:4], y_vars="NumStorePurchases")
sns.pairplot(df_transforme, x_vars=var_numeriques_reg[4:8], y_vars="NumStorePurchases")
sns.pairplot(df_transforme, x_vars=var_numeriques_reg[8:12], y_vars="NumStorePurchases")
sns.pairplot(df_transforme, x_vars=var_numeriques_reg[12:], y_vars="NumStorePurchases")

In [ ]:
for x in X[var_numeriques_reg].columns:
    corr = pearsonr(X[x], Y)
    print(
        "Le coefficient de corrélation linéaire entre Y et",
        x,
        "vaut",
        corr[0],
        "la pvalue vaut",
        corr[1],
    )

In [ ]:
# TODO: faire un joli affichage avec SNS heatmap

On décide déjà d'écarter la variable `Recency` de la régression.

In [ ]:
del X["Recency"]
var_numeriques_reg.remove("Recency")

### Réalisation du modèle

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
model_multiple = LinearRegression().fit(X_train, Y_train)
Y_train_hat = model_multiple.predict(X_train)

In [ ]:
plt.scatter(Y_train, Y_train_hat, color="black")
plt.title("Valeurs prédites contres vraies valeurs (donnéees d'entraînement)")
abline(1, 0)

### Hypothèse 2 : exogénéité sur les variables numériques
(Cela n'a pas de sens de tester l'exogénéité sur les variables catégoriques car la notion de corrélation ne fonctionne pas avec celles-ci.)

In [ ]:
residuals = Y_train - Y_train_hat

In [ ]:
for x in X_train[var_numeriques_reg].columns:
    corr = pearsonr(X_train[x], residuals)
    print(
        "Le coefficient de corrélation entre",
        x,
        "et les residus vaut",
        corr[0],
        "et la pvalue associée vaut",
        corr[1],
    )

Il n'y a pas d'endogénéité.

### Hypothèse 3 : l'erreur suit une loi normale centrée i.e. E(ε_i) = 0

In [ ]:
plt.hist(residuals, density=True)

x = np.linspace(-6, 6, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))
plt.title("Histogramme des résidus en superposition avec la densité de la loi normale")
plt.show()

In [ ]:
print("La moyenne des résidus vaut", statistics.mean(residuals))

In [ ]:
sm.qqplot(residuals, line="45")
print("On constate sur le qqplot que les points ne suivent pas la droite x = y")

In [ ]:
print(
    "Un test de shapiro, pour tester l'hypothèse de normalité, nous donne une pvalue de",
    stats.shapiro(residuals)[1],
    ". On rejette l'hypothèse nulle et on conclut que les résidus ne suivent pas une loi normale.",
)

### Hypothèse 4 : homoscédasticité

In [ ]:
plt.plot(residuals, "bo")
plt.title("Nuage de points des résidus")
abline(0, 6.5)
abline(0, -6.5)

In [ ]:
fit = sm.OLS(Y_train, sm.add_constant(X_train)).fit()

In [ ]:
fit.summary()

In [ ]:
res_names = ["F statistic", "p-value"]
gq_test = sms.het_goldfeldquandt(fit.resid, fit.model.exog)
lzip(res_names, gq_test)

In [ ]:
bp_test = sms.het_breuschpagan(fit.resid, fit.model.exog)
lzip(res_names, bp_test)

In [ ]:
white_test = sms.het_white(fit.resid, fit.model.exog)
lzip(res_names, white_test)

Le test de Goldfeld-Quandt, qui vérifie la constance de la variance entre deux échantillons, ne nous donne pas d'hétéroscédasticité, contrairement aux tests de White et de Breusch-Pagan.

### Hypothèse 5 : absence d'autocorrélation

In [ ]:
sm.graphics.tsa.plot_acf(residuals)
print("On remarque une absence d'autocorrélation.")

### Réduction du nombre de variables explicatives par le critère AIC

In [ ]:
def aic(X, Y, regressor=""):  # entrer X le dataframe des variables explicatives,
    # Y la variable expliquée, et le regresseur utilisé
    aic_list = []

    for col in X.columns:
        X_temp = X.drop(col, axis=1)

        if regressor == "linearOLS":
            model_temp = sm.OLS(Y, sm.add_constant(X_temp)).fit()
        elif regressor == "GLMpoisson":
            model_temp = sm.GLM(
                Y, sm.add_constant(X_temp), family=sm.families.Poisson()
            ).fit()

        else:
            return "Entrer un régresseur valide"

        aic_list.append([col, model_temp.aic])

    return aic_list

In [ ]:
def stepwise(X, Y, regressor=""):  # entrer X le dataframe des variables explicatives,
    # Y la variable expliquée, et le regresseur utilisé

    if regressor == "linearOLS":
        current_aic = (
            sm.OLS(Y, sm.add_constant(X)).fit().aic
        )  # aic avec les variables actuelles
    elif regressor == "GLMpoisson":
        current_aic = (
            sm.GLM(Y, sm.add_constant(X), family=sm.families.Poisson()).fit().aic
        )
    else:
        return "Entrer un régresseur valide"

    aic_list = aic(
        X, Y, regressor
    )  # liste des aic par supression des variables une a une

    my_bool = 0  # pour indiquer si aucune variable n'est a enlever
    for i in range(
        len(aic_list)
    ):  # si l'aic diminue pour une variable supprimee, on enleve cette variable du df X

        if aic_list[i][1] < current_aic:
            del X[aic_list[i][0]]  # a l'interieur des crochets c'est une string
            my_bool = 1

    if my_bool == 0:
        return "L'algorithme stepwise est terminé."

In [ ]:
print("Avant l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

In [ ]:
while True:
    if (
        stepwise(X_train, Y_train, regressor="linearOLS")
        == "L'algorithme stepwise est terminé."
    ):
        break

In [ ]:
stepwise(X_train, Y_train, regressor="linearOLS")

In [ ]:
print("Après l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

In [ ]:
# on refait les modèles après sélection de variables

X_test = X_test[
    X_train.columns
]  # homogénéisation des données de test et d'entraînement

fit = sm.OLS(Y_train, sm.add_constant(X_train)).fit()

model_multiple = LinearRegression().fit(X_train, Y_train)

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X_train.index, cooks[0])
plt.xlabel("Index des individus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.010) if x]
# détermine l'index dans la liste des individus dont la distance de cook dépasse 0.010

In [ ]:
for i in cooks_indexes:
    print(X_train.iloc[[i]].index[0])

In [ ]:
# TODO: supprimer les individus aux distances trop grandes

In [ ]:
influence_plot(fit)
print("")

Les distances de Cook ont été largement réduites par la sélection de variables au critère AIC.

### Score du modèle

#### Qualité d'ajustement

In [ ]:
print(f"Le R² du modèle vaut {model_multiple.score(X_train, Y_train)}")

In [ ]:
Y_train_hat = model_multiple.predict(X_train)

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model_multiple.score(X_train, Y_train)}"
)
print(f"R² du modèle sur les données de test = {model_multiple.score(X_test, Y_test)}")

In [ ]:
Y_test_hat = model_multiple.predict(X_test)

In [ ]:
plt.scatter(Y_test, Y_test_hat, color="black")
abline(1, 0)
plt.title("Valeurs prédites contre les vraies valeurs : données de test du modèle")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

## Regression par modèle linéaire généralisé (GLM) : régression de Poisson

- Notons $Y$ notre variable d'intérêt et $X_1, ..., X_P$ nos variables explicatives.  
- On suppose que Y suit une loi de Poisson de paramètre $\lambda \in \mathbb{R+*}$


- On cherche à déterminer les coefficients de l'équation :

$\log(\mathrm{E}(Y|X)) = \log( \lambda ) = \alpha + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_P X_P $

Ce qui revient à déterminer les coefficients de l'équation :

$\lambda = e^{\alpha + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_P X_P}$

L'objectif de la régression est d'estimer $\alpha, \beta_1, ..., \beta_P$.  
Une fois ces coefficients estimés, on peut déterminer, pour un nouveau vecteur $X$,  
$E(Y|X) = e^{\alpha + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_P X_P}$

### Hypothèses à vérifier : 

<br>
- Avant de choisir la régression de Poisson :

1. La variable d'intérêt doit être une variable de comptage, à valeurs entières positives.  

2. La variable d'intérêt doit suivre une loi de Poisson :  
$Y | X ∼ Poisson(\lambda)$, où $\lambda \in \mathbb{R+*}$ , en particulier, la moyenne de Y doit être égale à sa variance.

3. $log(\lambda)$ est une fonction linéaire de $X_1, X_2, ... , X_P$  

<br>
- A posteriori :

4. $Cov(X_i, ε_j) = 0$, pour tout $i, j \in N \times N, i \neq j$ (exogénéité) et pour chaque variable explicative $X_p$

5. L'erreur suit une loi normale centrée i.e. $E(ε_i) = 0$

6. L’erreur est de variance constante (homoscédasticité)
i.e $Var(ε_i) = \sigma^2$, une constante

7. Les erreurs sont indépendantes (absence d'autocorrélation)
i.e. $Cov(εi, εj) = 0$, pour tout $i, j \in N \times N, i \neq j$

8. Absence de colinéarité entre les variables explicatives,
i.e. $X^t * X$ est régulière, $det(X^t * X) \neq 0$

In [ ]:
X = df_reg[var_categoriques_reg + var_numeriques_reg]
Y = df["NumStorePurchases"]

### Hypothèses 1 : nature de la variable d'intérêt
La variable `NumberStorePurchases` désigne le nombre d'achat effectués en magasin. C'est bien une variable de comptage dont les valeurs sont entières et positives.

### Hypothèse 2 : loi de Poisson ?

In [ ]:
from scipy.stats import poisson

In [ ]:
plt.hist(df["NumStorePurchases"], density=True)

# creating a numpy array for x-axis
x = np.arange(0, 20, 1)

# poisson distribution data for y-axis
y = poisson.pmf(x, mu=5)

plt.plot(x, y)
plt.plot()

In [ ]:
from scipy.stats import kstest

In [ ]:
ks_statistic, p_value = kstest(df["NumStorePurchases"], "poisson", args=(5, 0))
print(ks_statistic, p_value)

In [ ]:
print(
    "Variance de Y :",
    df["NumStorePurchases"].var(),
    ": Espérance de Y:",
    df["NumStorePurchases"].mean(),
)

In [ ]:
print(
    "La variance et l'espérance de Y ne sont pas égales. Cela peut dégrader la qualité de notre régression."
)

### Hypothèse 3 : lien linéaire entre $log(Y)$ et $X_1, ..., X_P$

In [ ]:
null_index = Y[Y == 0].index
# on élimine les 0 pour appliquer le log

In [ ]:
sns.pairplot(
    pd.concat(
        [X.drop(index=null_index, axis=0), np.log(Y.drop(index=null_index, axis=0))],
        axis=1,
    ),
    x_vars=var_numeriques_reg[0:4],
    y_vars="NumStorePurchases",
)
sns.pairplot(
    pd.concat(
        [X.drop(index=null_index, axis=0), np.log(Y.drop(index=null_index, axis=0))],
        axis=1,
    ),
    x_vars=var_numeriques_reg[4:8],
    y_vars="NumStorePurchases",
)
sns.pairplot(
    pd.concat(
        [X.drop(index=null_index, axis=0), np.log(Y.drop(index=null_index, axis=0))],
        axis=1,
    ),
    x_vars=var_numeriques_reg[8:12],
    y_vars="NumStorePurchases",
)
sns.pairplot(
    pd.concat(
        [X.drop(index=null_index, axis=0), np.log(Y.drop(index=null_index, axis=0))],
        axis=1,
    ),
    x_vars=var_numeriques_reg[12:],
    y_vars="NumStorePurchases",
)

In [ ]:
for x in X[var_numeriques_reg].columns:
    corr = pearsonr(
        X.drop(index=null_index, axis=0)[x], np.log(Y.drop(index=null_index, axis=0))
    )
    print(
        "Le coefficient de corrélation linéaire entre log(Y) et",
        x,
        "vaut",
        corr[0],
        "la pvalue vaut",
        corr[1],
    )

Il existe bien un lien linéaire entre $log(Y)$ et les variables explicatives.

### Réalisation du modèle

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import PoissonRegressor

In [ ]:
model_poisson = PoissonRegressor().fit(X_train, Y_train)
Y_train_hat = model_poisson.predict(X_train)

Ici, notre fonction de lien est $f(\lambda) = log(\lambda)$  
Elle est utilisée par défaut par sci-kit learn 

In [ ]:
plt.scatter(Y_train, Y_train_hat, color="black")
plt.title("Valeurs prédites contres vraies valeurs (donnéees d'entraînement)")
abline(1, 0)

### Hypothèse 4 : exogénéité sur les variables numériques
(Cela n'a pas de sens de tester l'exogénéité sur les variables catégoriques car la notion de corrélation ne fonctionne pas avec celles-ci.)

In [ ]:
residuals = Y_train - Y_train_hat

In [ ]:
for x in X_train[var_numeriques_reg].columns:
    corr = pearsonr(X_train[x], residuals)
    print(
        "Le coefficient de corrélation entre",
        x,
        "et les residus vaut",
        corr[0],
        "et la pvalue associée vaut",
        corr[1],
    )

Il n'y a pas d'endogénéité.

### Hypothèse 5 : l'erreur suit une loi normale centrée i.e. $E(ε_i) = 0$

In [ ]:
plt.hist(residuals, density=True)

x = np.linspace(-6, 6, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1))
plt.title("Histogramme des résidus en superposition avec la densité de la loi normale")
plt.show()

In [ ]:
print("La moyenne des résidus vaut", statistics.mean(residuals))

In [ ]:
sm.qqplot(residuals, line="45")
print("On constate sur le qqplot que les points ne suivent pas la droite x = y")

In [ ]:
print(
    "Un test de shapiro, pour tester l'hypothèse de normalité, nous donne une pvalue de",
    stats.shapiro(residuals)[1],
    ". On rejette l'hypothèse nulle et on conclut que les résidus ne suivent pas une loi normale.",
)

### Hypothèse 6 : homoscédasticité

In [ ]:
plt.plot(residuals, "bo")
plt.title("Nuage de points des résidus")
abline(0, 6.5)
abline(0, -6.5)

In [ ]:
fit = sm.GLM(Y_train, sm.add_constant(X_train), family=sm.families.Poisson()).fit()

In [ ]:
fit.summary()

In [ ]:
res_names = ["F statistic", "p-value"]
gq_test = sms.het_goldfeldquandt(fit.resid_response, fit.model.exog)
lzip(res_names, gq_test)

La pvalue du test de Goldfeld-Quandt est supérieure à 0.05 dont on conserve l'hypothèse d'homoscédasticité au seuil de 5% (et même au delà).

### Hypothèse 7 : absence d'autocorrélation

In [ ]:
sm.graphics.tsa.plot_acf(residuals)
print("On remarque une absence d'autocorrélation.")

### Réduction du nombre de variables explicatives par le critère AIC

In [ ]:
print("Avant l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

In [ ]:
while True:
    if (
        stepwise(X_train, Y_train, regressor="GLMpoisson")
        == "L'algorithme stepwise est terminé."
    ):
        break

In [ ]:
stepwise(X_train, Y_train, regressor="GLMpoisson")

In [ ]:
print("Après l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

In [ ]:
# on refait les modèles après sélection de variables

X_test = X_test[
    X_train.columns
]  # homogénéisation des données de test et d'entraînement

fit = sm.GLM(Y_train, sm.add_constant(X_train), family=sm.families.Poisson()).fit()

model_poisson = PoissonRegressor().fit(X_train, Y_train)

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X_train.index, cooks[0])
plt.xlabel("Index des individus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.010) if x]
# détermine l'index dans la liste des individus dont la distance de cook dépasse 0.010

In [ ]:
for i in cooks_indexes:
    print(X_train.iloc[[i]].index[0])

In [ ]:
# TODO: supprimer les individus aux distances trop grandes
# Remarque : statsmodels ne fait pas d'influence plot pour les modeles GLM

### Score du modèle

#### Qualité d'ajustement

In [ ]:
print(f"Le R² du modèle vaut {model_poisson.score(X_train, Y_train)}")

In [ ]:
Y_train_hat = model_poisson.predict(X_train)

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model_poisson.score(X_train, Y_train)}"
)
print(f"R² du modèle sur les données de test = {model_poisson.score(X_test, Y_test)}")

In [ ]:
Y_test_hat = model_poisson.predict(X_test)

In [ ]:
plt.scatter(Y_test, Y_test_hat, color="black")
abline(1, 0)
plt.title("Valeurs prédites contre les vraies valeurs : données de test du modèle")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

In [ ]:
print("Constante : ", model_poisson.intercept_)
for i in range(len(model_poisson.coef_)):
    print(f" beta_{i+1} = {model_poisson.coef_[i]} ")

## Régression polynomiale

### Création des données

On conserve les mêmes variables explicatives et la même variable d'intérêt, mais en séparant les variables catégoriques des variables numériques qu'on transformera en monômes.  
Ceci car cela n'aurait pas de sens d'élever des variables catégoriques au carré et afin d'éviter l'explosion combinatoire.

In [ ]:
X_ref = df_reg[var_numeriques_reg + var_categoriques_reg]

In [ ]:
X_numeriques = df_reg[var_numeriques_reg]
X_categoriques = df_reg[var_categoriques_reg]
Y = df_reg["NumStorePurchases"]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# Transformer les données en matrice de caractéristiques polynomiales
poly = PolynomialFeatures(degree=2)

X_poly = poly.fit_transform(X_numeriques)
# Remarque : pendant la transformation, l'ordre des lignes est conservé, ce qui permettra de remettre les index

In [ ]:
# trouvé sur stack over flow :
# permet de créer un df à partir de l'objet numpy array en sortie de la fonction fit_transform de sklearn

target_feature_names = [
    "x".join(["{}^{}".format(pair[0], pair[1]) for pair in tuple if pair[1] != 0])
    for tuple in [zip(X.columns, p) for p in poly.powers_]
]
# détermine les noms de colonnes


X_poly = pd.DataFrame(X_poly, columns=target_feature_names)

In [ ]:
# je remets les indexes au df X_poly afin de pouvoir le concaténer avec le df des var categoriques avant la régression
X_poly = X_poly.set_index(pd.Index(X_categoriques.index))

In [ ]:
X = pd.concat([X_poly, X_categoriques], axis=1)

In [ ]:
X.shape

Bien évidemment, il y a de la redondance dans les variables explicatives et donc de la colinéarité.

### Réalisation du modèle

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
model_poly = LinearRegression().fit(X_train, Y_train)
Y_train_hat = model_poly.predict(X_train)

In [ ]:
plt.scatter(Y_train, Y_train_hat, color="black")
plt.title("Valeurs prédites contres vraies valeurs (donnéees d'entraînement)")
abline(1, 0)

### Réduction du nombre de variables explicatives par le critère AIC

In [ ]:
print("Avant l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

In [ ]:
while True:
    if (
        stepwise(
            X_train, list(Y_train), regressor="linearOLS"
        )  # il faut transformer Y en liste sinon cela bug, je ne sais pas pourquoi
        == "L'algorithme stepwise est terminé."
    ):
        break

In [ ]:
stepwise(X_train, list(Y_train), regressor="linearOLS")

In [ ]:
print("Après l'AIC, le nombre de variables explicatives est :", X_train.shape[1])

La procédure AIC a permi de purifier notre modèle de 77 variables explicatives.

In [ ]:
# on refait les modèles après sélection de variables

X_test = X_test[
    X_train.columns
]  # homogénéisation des données de test et d'entraînement

fit = sm.OLS(
    list(Y_train), sm.add_constant(X_train)
).fit()  # comme il existe deja une constante dans X_train, statsmodels n'en rajoute pas une deuxieme

model_poly = LinearRegression().fit(X_train, Y_train)

### Distance de Cook

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X_train.index, cooks[0])
plt.xlabel("Index des individus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
cooks_indexes = [i for i, x in enumerate(cooks[0] > 0.02) if x]
# détermine l'index dans la liste des individus dont la distance de cook dépasse 0.010

In [ ]:
extreme_ind_list = [X_train.iloc[[i]].index[0] for i in cooks_indexes]

for ind in extreme_ind_list:
    print(ind)

In [ ]:
influence_plot(fit)
print("")

In [ ]:
X_train.drop(extreme_ind_list + [6237, 9058], inplace=True)
Y_train.drop(extreme_ind_list + [6237, 9058], inplace=True)

In [ ]:
# on refait les modèles après suppression des individus extremes

X_test = X_test[
    X_train.columns
]  # homogénéisation des données de test et d'entraînement

fit = sm.OLS(
    list(Y_train), sm.add_constant(X_train)
).fit()  # comme il existe deja une constante dans X_train, statsmodels n'en rajoute pas une deuxieme

model_poly = LinearRegression().fit(X_train, Y_train)

In [ ]:
influence = fit.get_influence()
cooks = influence.cooks_distance

In [ ]:
plt.scatter(X_train.index, cooks[0])
plt.xlabel("Index des individus")
plt.ylabel("Distances de Cook")
plt.show()

In [ ]:
influence_plot(fit)
print("")

### Score du modèle

#### Qualité d'ajustement

In [ ]:
print(f"Le R² du modèle vaut {model_poly.score(X_train, Y_train)}")

In [ ]:
Y_train_hat = model_poly.predict(X_train)

In [ ]:
print(f"MSE = {mean_squared_error(Y_train, Y_train_hat)}")
print(f"RMSE = {mean_squared_error(Y_train, Y_train_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_train, Y_train_hat)}")

#### Qualité de prédiction

Train / test split

In [ ]:
print(
    f"R² du modèle sur les données d'entraînement = {model_poly.score(X_train, Y_train)}"
)
print(f"R² du modèle sur les données de test = {model_poly.score(X_test, Y_test)}")

In [ ]:
Y_test_hat = model_poly.predict(X_test)

In [ ]:
plt.scatter(Y_test, Y_test_hat, color="black")
abline(1, 0)
plt.title("Valeurs prédites contre les vraies valeurs : données de test du modèle")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test, Y_test_hat)}")
print(f"RMSE = {mean_squared_error(Y_test, Y_test_hat, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test, Y_test_hat)}")

La mesure de la qualité d'ajustement à nos données de test est biaisée par des valeurs prédites extrêmes.

In [ ]:
sns.histplot(Y_test_hat)

In [ ]:
Y_test_hat_new = Y_test_hat[Y_test_hat > 0]
Y_test_new = Y_test[Y_test_hat > 0]

In [ ]:
plt.scatter(Y_test_new, Y_test_hat_new, color="black")
abline(1, 0)
plt.title("Valeurs prédites contre les vraies valeurs : données de test du modèle")

In [ ]:
print(f"MSE = {mean_squared_error(Y_test_new, Y_test_hat_new)}")
print(f"RMSE = {mean_squared_error(Y_test_new, Y_test_hat_new, squared=False)}")
print(f"MAE = {mean_absolute_error(Y_test_new, Y_test_hat_new)}")

Corrigé de ces erreurs, le modèle polynomial est notre meilleur modèle.